In [1]:
pip install linearmodels

In [0]:
import pandas as pd
from linearmodels import IVGMM

In [3]:
file="https://github.com/VitorKamada/ECO7110/raw/master/Data/card.dta"
df = pd.read_stata(file)
df.head()

,id,nearc2,nearc4,educ,age,fatheduc,motheduc,weight,momdad14,sinmom14,step14,reg661,reg662,reg663,reg664,reg665,reg666,reg667,reg668,reg669,south66,black,smsa,south,smsa66,wage,enroll,kww,iq,married,libcrd14,exper,lwage,expersq
0,2,0,0,7,29,NaN,NaN,158413.0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,1,548,0,15.0,NaN,1.0,0.0,16,6.306275,256
1,3,0,0,12,27,8.0,8.0,380166.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,481,0,35.0,93.0,1.0,1.0,9,6.175867,81
2,4,0,0,12,34,14.0,12.0,367470.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,721,0,42.0,103.0,1.0,1.0,16,6.580639,256
3,5,1,1,11,27,11.0,12.0,380166.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,250,0,25.0,88.0,1.0,1.0,10,5.521461,100
4,6,1,1,12,34,8.0,7.0,367470.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,729,0,34.0,108.0,1.0,0.0,16,6.591674,256


In [0]:
df['const'] = 1
Demo = ['const', 'black', 'exper', 'expersq', 
        'south','smsa','smsa66']
Region = ['reg662', 'reg663', 'reg664', 'reg665', 
          'reg666', 'reg667', 'reg668','reg669']
df['blackeduc'] = df['educ']*df['black']
df['blacknearc4'] = df['black']*df['nearc4']

In [5]:
# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

print(df[['lwage','educ','nearc4'] + 
           Demo].describe(percentiles=[]))

         lwage     educ   nearc4    const  ...  expersq    south     smsa   smsa66
count 3,010.00 3,010.00 3,010.00 3,010.00  ... 3,010.00 3,010.00 3,010.00 3,010.00
mean      6.26    13.26     0.68     1.00  ...    95.58     0.40     0.71     0.65
std       0.44     2.68     0.47     0.00  ...    84.62     0.49     0.45     0.48
min       4.61     1.00     0.00     1.00  ...     0.00     0.00     0.00     0.00
50%       6.29    13.00     1.00     1.00  ...    64.00     0.00     1.00     1.00
max       7.78    18.00     1.00     1.00  ...   529.00     1.00     1.00     1.00

[6 rows x 10 columns]


In [6]:
OLS = IVGMM(df.lwage, df[['educ']+['blackeduc']+Demo+Region],
            None, None).fit()
print(OLS)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.3017
Estimator:                        OLS   Adj. R-squared:                 0.2980
No. Observations:                3010   F-statistic:                    1404.0
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        18:23:02   Distribution:                 chi2(16)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ           0.0708     0.0039     18.070     0.0000      0.0631      0.0785
blackeduc      0.0179     0.0060     2.9607     0.00

In [7]:
IV = IVGMM(df.lwage, df[Demo+Region], df[['educ','blackeduc']],
           df[['nearc4','blacknearc4']]).fit()
print(IV)

                          IV-GMM Estimation Summary                           
Dep. Variable:                  lwage   R-squared:                      0.2435
Estimator:                     IV-GMM   Adj. R-squared:                 0.2395
No. Observations:                3010   F-statistic:                    842.36
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        18:23:02   Distribution:                 chi2(16)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          3.7346     0.9410     3.9688     0.0001      1.8903      5.5790
black         -0.2828     0.4998    -0.5658     0.57

In [8]:
First1 = IVGMM(df.educ, df[['nearc4']+['blacknearc4']+Demo+Region],
               None, None).fit()
df['Res1'] = First1.resids
print(First1)

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.4771
Estimator:                        OLS   Adj. R-squared:                 0.4743
No. Observations:                3010   F-statistic:                    3697.3
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        18:23:02   Distribution:                 chi2(16)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
nearc4          0.3192     0.0956     3.3391     0.0008      0.1318      0.5065
blacknearc4     0.0030     0.1695     0.0176    

In [9]:
First2 = IVGMM(df.blackeduc, df[['nearc4']+['blacknearc4']+Demo+Region],
               None, None).fit()
df['Res2'] = First2.resids
print(First2)

                            OLS Estimation Summary                            
Dep. Variable:              blackeduc   R-squared:                      0.9516
Estimator:                        OLS   Adj. R-squared:                 0.9514
No. Observations:                3010   F-statistic:                 2.365e+04
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        18:23:02   Distribution:                 chi2(16)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
nearc4         -0.0909     0.0327    -2.7798     0.0054     -0.1550     -0.0268
blacknearc4     0.8747     0.1806     4.8445    

In [10]:
Exog = df[['educ']+['blackeduc']+['Res1']+['Res2']+Demo+Region]
CF = IVGMM(df.lwage, Exog, None, None).fit()
print(CF)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.3020
Estimator:                        OLS   Adj. R-squared:                 0.2978
No. Observations:                3010   F-statistic:                    1407.0
Date:                Tue, Aug 06 2019   P-value (F-stat)                0.0000
Time:                        18:23:02   Distribution:                 chi2(18)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
educ           0.1274     0.0536     2.3766     0.0175      0.0223      0.2324
blackeduc      0.0109     0.0380     0.2871     0.77